## Orchestrate the nested chat

In [1]:
## import required libraries
import os
from autogen import AssistantAgent, ConversableAgent, UserProxyAgent
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
## llm configuration
llm_config = {
    "model": "gpt-4o",
    "api_key": os.getenv("OPENAI_API_KEY")
}

## Writer Agent

In [3]:
writer = AssistantAgent(
    name="Writer",
    system_message="""
        You are a writer. You write engaging and concise blogpost (with title) on given topics. 
        You must polish your writing based on the feedback you receive and give a refined version.
        Only return your final work without additional comments.
    """,
    llm_config=llm_config,
)

## Task

In [4]:
task = "Write a concise but engaging blogpost about GPUs. Make sure the blogpost is within 100 words."

In [5]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [6]:
print(reply)

**The Powerhouse of Modern Computing: GPUs**

Graphics Processing Units (GPUs) have revolutionized the digital landscape beyond just rendering video game graphics. Today’s GPUs, with their parallel processing capabilities, are the unseen architects behind AI breakthroughs, cryptocurrency mining, and high-performance scientific simulations. Efficiently handling multiple tasks simultaneously, GPUs accelerate tasks that would overwhelm traditional CPUs. As technology evolves, GPUs continue to reshape industries, from powering immersive virtual reality experiences to enabling real-time data analytics. With companies like NVIDIA and AMD leading advancements, GPUs symbolize the relentless drive towards faster, smarter computing, driving innovation in surprising and transformative ways.


## Simple review of the post

In [7]:
## reviewer agent
reviewer = AssistantAgent(
    name="Reviewer",
    is_termination_msg=lambda x:x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="""You are a reviewer. you review the work of
                    the writer and provide constructive feedback to help imporve the quality of the content.
    """,
)

In [8]:
res = reviewer.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Reviewer (to Writer):

Write a concise but engaging blogpost about GPUs. Make sure the blogpost is within 100 words.

--------------------------------------------------------------------------------
Writer (to Reviewer):

**Harnessing the Power of GPUs: Revolutionizing Tech Landscapes**

Graphics Processing Units (GPUs) have evolved from being niche components for gaming to pivotal drivers of technological innovation. Capable of processing parallel tasks efficiently, GPUs excel in data-intensive fields like artificial intelligence, machine learning, and data science. By offloading demanding tasks from CPUs, they unlock unprecedented speeds and capabilities. Today, industries leverage GPU power for tasks from autonomous vehicles to large-scale scientific computations. As technology advances, GPUs continue to push boundaries, offering future-ready solutions that satisfy our ever-growing computing needs, proving their indispensable role in shaping the future of digital landscapes.

------

## Adding multi-level reviews using nested chats

In [22]:
SEO_reviewer = AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="As an SEO expert, your role is to analyze and enhance content for optimal search engine performance. "
        "Focus on providing actionable recommendations that boost rankings and drive organic traffic. "
        "Limit your feedback to 3 key points, ensuring they are specific and directly applicable. "
        "Start each review by introducing yourself as an SEO Reviewer.",
    
#     """As and SEO expert, your role is to analyze and enhence content for optimal search engine performance.
#             Focus on providing actionable recommendations that boost rankings and drive organic traffic.
#             Limit your feedback to 3 key points, ensuring they are specific and directly applicable.
#             Start each review by introducing yourself as an SEO Reviewer."""
)

In [23]:
grammatical_error_reviewer = AssistantAgent(
    name="Grammatical_Error_Reviewer",
    llm_config=llm_config,
    system_message="As a grammar specialist, your task is to meticulously examine content "
        "for grammatical errors, punctuation mistakes, and style inconsistencies. "
        "Provide up to 3 key points addressing the most significant grammatical issues. "
        "Ensure your feedback is specific, actionable, and includes examples where appropriate. "
        "Begin each review by introducing yourself as a Grammatical Error Reviewer.",
    
#     """As a grammer specialist, your task is to meticulously examine content for grammatical errors,
#             punctuation mistakes, and style inconsistencies. Provide up to 3 key points addressing the most 
#             significant grammatical issues. Ensure your feed back is specific, actionable, and includes examples 
#             where appropriate. Begin each review by introducing yourself as Grammatical Error Reviewer.""",
)

In [27]:
ethics_reviewer = AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="As an ethics specialist, your role is to evaluate content for ethical integrity "
        "and identify any potential moral concerns. "
        "Provide up to 3 specific, actionable recommendations to address ethical issues. "
        "Ensure your feedback is concise and directly applicable. "
        "Start each review by introducing yourself as an Ethics Reviewer.",
    
#     """As an ethics specialist, your role is to evaluate content for ethical integrity and identity 
#             any potential moral concerns. Provide up to 3 specific, actionable recommendations to address ethical 
#             issues. Ensure your feedback is concise and directly applicable. Start each review by introducing yourself
#             as an Ethics Reviewer.""",
)

In [25]:
meta_reviewer = AssistantAgent(
    name="Meta_Reviewer",
    llm_config = llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
    
#     """You are a meta reviewer, you aggragate and review the work of other reviewers and give a final suggestion on the content.""",
    
)

## Orchestrating the nested chat

In [52]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
    "recipient": SEO_reviewer,
    "message": reflection_message,
    "summary_method": "reflection_with_llm",
    "summary_args": {"summary_prompt" : 
                     " Return review into as JSON object only:"
                     "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
    "max_turns": 1},
    {
        "recipient": grammatical_error_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt" : 
                         " Return review into as JSON object only:"
                         "{'Reviewer': '', 'Review': ''}.",},
        "max_turns": 1
    },
    {
        "recipient": ethicts_reviewer,
        "message": reflection_message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt" : 
                         " Return review into as JSON object only:"
                         " {'Reviewer': '', 'Review': ''}",},
        "max_turns": 1
    },
    {
        "recipient": meta_reviewer,
        "message": "Aggregate feedback from all reviewers and give final suggestions on the writing.",
        "max_turns": 1},
]

In [53]:
reviewer.register_nested_chats(
    review_chats,
    trigger=writer
)

In [54]:
res = reviewer.initiate_chat(
    recipient = writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Reviewer (to Writer):

Write a concise but engaging blogpost about GPUs. Make sure the blogpost is within 100 words.

--------------------------------------------------------------------------------
Writer (to Reviewer):

**The Powerhouse of Computing: Unleashing the Potential of GPUs**

In the rapidly evolving realm of technology, Graphics Processing Units (GPUs) have revolutionized the way we compute, design, and innovate. Originally built to enhance gaming visuals, GPUs now drive breakthroughs across industries, from artificial intelligence to scientific simulations. Their parallel processing power enables massive data handling and accelerates machine learning algorithms, making them indispensable in modern applications. With tech giants racing to enhance GPU capabilities, we stand on the brink of unprecedented advancements. Embrace the future as GPUs redefine computing—where creativity meets efficiency at lightning speed. Welcome to the era of unparalleled possibilities.

---------


--------------------------------------------------------------------------------
Ethics_Reviewer (to Reviewer):

Ethics Reviewer:

The content primarily discusses the role and advancements of Graphics Processing Units (GPUs) within the tech industry. Upon reviewing for ethical concerns, the article upholds a neutral informative stance, focusing on technological innovation without apparent biases or moral conflicts. However, as technology impacts society significantly, even technical articles could include broader ethical considerations. Here are some recommendations:

1. **Address Environmental Impact:**
   - While highlighting GPU advancements, include a section on their environmental footprint. GPUs, like other tech hardware, contribute to energy consumption and electronic waste. Encouraging sustainable practices and discussing efforts to mitigate these impacts can promote ethical awareness.

2. **Consider Societal Implications:**
   - Expand the discussion to reflect on the societa


--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (ef8e0db5-93a2-4987-b63c-c0ba9cca2391): Maximum turns (1) reached
Reviewer (to Writer):

Based on the aggregate feedback from the reviewers, here are the final suggestions to enhance the content:

### SEO Recommendations
1. **Incorporate Target Keywords**: It is crucial to sprinkle relevant keywords throughout the content judiciously. Phrases such as "GPU advancements," "parallel processing power," and "machine learning acceleration" should be included to target specific search intents and improve the discoverability of the article.

2. **Optimize for Featured Snippets**: Structure the content to qualify for Google's featured snippets by including concise lists or tables that summarize key information. This approach can increase the likelihood of better visibility and click-through rates by standing out in search engine results pages (SERPs).

3. **Add Internal and External Links

## Get summary

In [55]:
print(res.summary)

**The Powerhouse of Computing: Unleashing the Potential of GPUs**

In today's tech-driven world, Graphics Processing Units (GPUs) are pivotal in transforming computing landscapes. Initially designed for gaming graphics, GPUs now spearhead innovations, driving GPU advancements in artificial intelligence and accelerating machine learning algorithms. Their parallel processing power manages vast data with remarkable efficiency. Amidst growing concerns about energy consumption and electronic waste, the industry explores sustainable solutions to minimize environmental impact. As we harness this technology, it's crucial to address societal implications such as job displacement and privacy concerns. The future of computing lies in GPUs, where technology meets ethical responsibility and creativity flourishes.
